In [19]:
!pip install antigranular &> /dev/null

In [20]:
import antigranular as ag
session = ag.login(<client_id>,<client_secret>, competition = "Fraud Detection Hackathon with Privacy Village #DPD25FEST")

Dataset "Transaction Fraud Hackathon Dataset" loaded to the kernel as transaction_fraud_hackathon_dataset
Key Name                       Value Type     
---------------------------------------------
test_x                         DataFrame      
train_x                        PrivateDataFrame
train_y                        PrivateDataFrame

Connected to Antigranular server session id: a8551efd-c932-4f32-9b37-8643d99132ea, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


In [22]:
%%ag
x_train = transaction_fraud_hackathon_dataset["train_x"]
y_train = transaction_fraud_hackathon_dataset["train_y"]
x_test = transaction_fraud_hackathon_dataset["test_x"]

In [21]:
session.privacy_odometer()

{'total_epsilon_used': 4.595457271043286,
 'total_delta_used': 4.4000000000000006e-05,
 'library_costs': {'op_tensorflow': {'total_delta': 4.4000000000000006e-05,
   'total_epsilon': 2.595457271043286,
   'total_requests': 8},
  'op_pandas': {'total_delta': 0.0,
   'total_epsilon': 2.0,
   'total_requests': 2}},
 'dataset_costs': {'Transaction Fraud Hackathon Dataset': {'delta': 4.4000000000000006e-05,
   'eps': 4.595457271043286}}}

In [26]:
%%ag
import tensorflow as tf
from op_pandas import standard_scaler, PrivateDataFrame
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from op_tensorflow import PrivateKerasModel, PrivateDataLoader

# Modified Sequential model with better calibration
seqM = Sequential([
    # Input block with gradual feature processing
    Dense(32, input_shape=(9,)),
    BatchNormalization(momentum=0.9),  # Higher momentum for stability
    tf.keras.layers.ReLU(),
    Dropout(0.1),  # Reduced dropout for more stable predictions

    # Intermediate processing with careful regularization
    Dense(32),
    BatchNormalization(momentum=0.9),
    tf.keras.layers.ReLU(),
    Dropout(0.1),

    # Gradual reduction in dimensionality
    Dense(16),
    BatchNormalization(momentum=0.9),
    tf.keras.layers.ReLU(),

    # Output block with additional stability layers
    Dense(8),
    BatchNormalization(momentum=0.9),
    tf.keras.layers.ReLU(),

    # Final layer with adjusted activation
    Dense(1, activation='sigmoid')
])

# Adjusted DP model with more conservative privacy settings
dp_model = PrivateKerasModel(
    model=seqM,
    l2_norm_clip=0.7,  # Reduced for better stability
    noise_multiplier=0.7  # Reduced to help with calibration
)

# Modified optimizer with more conservative settings
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.0008,  # Reduced for more stable convergence
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-7,  # Smaller epsilon for better numerical stability
)


# Compile model
dp_model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [28]:
%%ag
data_loader = PrivateDataLoader(feature_df=x_train , label_df=y_train, batch_size=32)

In [30]:
%%ag
dp_model.fit(x=data_loader, epochs=35,shuffle=True )

Epoch 1/35

1094/1094 - 38s - loss: 0.6505 - accuracy: 0.6247 - 38s/epoch - 34ms/step

Epoch 2/35

1094/1094 - 29s - loss: 0.6231 - accuracy: 0.6533 - 29s/epoch - 27ms/step

Epoch 3/35

1094/1094 - 29s - loss: 0.6176 - accuracy: 0.6605 - 29s/epoch - 27ms/step

Epoch 4/35

1094/1094 - 29s - loss: 0.6123 - accuracy: 0.6647 - 29s/epoch - 27ms/step

Epoch 5/35

1094/1094 - 30s - loss: 0.6075 - accuracy: 0.6711 - 30s/epoch - 27ms/step

Epoch 6/35

1094/1094 - 29s - loss: 0.6066 - accuracy: 0.6681 - 29s/epoch - 27ms/step

Epoch 7/35

1094/1094 - 29s - loss: 0.6078 - accuracy: 0.6694 - 29s/epoch - 27ms/step

Epoch 8/35

1094/1094 - 29s - loss: 0.6077 - accuracy: 0.6680 - 29s/epoch - 26ms/step

Epoch 9/35

1094/1094 - 29s - loss: 0.6006 - accuracy: 0.6772 - 29s/epoch - 26ms/step

Epoch 10/35

1094/1094 - 29s - loss: 0.5988 - accuracy: 0.6809 - 29s/epoch - 27ms/step

Epoch 11/35

1094/1094 - 30s - loss: 0.6021 - accuracy: 0.6736 - 30s/epoch - 27ms/step

Epoch 12/35

1094/1094 - 30s - loss: 0.60

In [31]:
%%ag
test=PrivateDataFrame(x_test).copy()
y_pred = dp_model.predict(test, label_columns=["output"])

469/469 [==============================] - 4s 7ms/step



In [32]:
%%ag
# Note that the predictions are a float scalar, so we scale it!
def f(x: float) -> float:
    return 1 if x > 0.5 else 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))

In [33]:
%%ag
result = submit_predictions(y_pred)

score: {'leaderboard': 0.67163529482383, 'logs': {'BIN_ACC': 0.6946125811790465, 'LIN_EPS': -0.022977286355216432}}

